# Mutation-Based Fuzzing

## Fuzzing a URL Parser

In [2]:
from ipynb.fs.full.fuzzer import *

C:\Users\hxie\AppData\Local\Temp\tmpo29hjp8e\input.txt
6.>:39.$42"=00$<!'9&0$4*!-.1'35=".?2*21-7== #<7+-8#;8 <=6<??$%% 9"27!3:9#0"&(3&46*2::8=:='2;?95$#6!
CompletedProcess(args=['C:\\Windows\\System32\\calc.exe', 'C:\\Users\\hxie\\AppData\\Local\\Temp\\tmpo29hjp8e\\input.txt'], returncode=0, stdout='', stderr='')
Some input
=(-*14:>?>0* 6*<339-
4&?5:761:+;.5?$-2?!3
#%1'>0,"";+3';!"0%$:
5#+7</)5/"?"?0$8&67.
0!'*&!-2"5?6*"'9#=2 
&%0(?&"8%','98>> 7-:
4&#=:8&3'6(7.%*/!($1
01:')!< (65&$65 $9);
)33('.-1.*6/3'$"*8'$
'1)< ,$!%3  %<(>>;1/


In [4]:
fuzzer()

'/,0!26,53%#16%3- ;:>=8\'=9\'43();9*6"6/0\'5-?3* $%>=9:+\'=:;:5(< 84<;<\'<+!7"?\'<&;=+0  5( -46'

In [5]:
from typing import Tuple, List, Callable, Any, Set

In [6]:
from urllib.parse import urlparse

In [7]:
urlparse("http://www.google.com/search?q=fuzzing")

ParseResult(scheme='http', netloc='www.google.com', path='/search', params='', query='q=fuzzing', fragment='')

In [8]:
def http_program(url: str) -> bool:
    supported_schemes = ["http", "https"]
    result = urlparse(url)
    if result.scheme not in supported_schemes:
        raise ValueError("Scheme must be one of " + 
                         repr(supported_schemes))
    if result.netloc == '':
        raise ValueError("Host must be non-empty")

    # Do something with the URL
    return True

In [9]:
fuzzer(char_start=32, char_range=96)

't{z*R{\x7fmjf7~/E:uxD*}kr.y@vj[H\x7f!So>`wJ86<~zE2yQ3Th=z?\x7fixg~T:ln_[%t3HA@eIi((dY5Sb$o.;i'

In [10]:
for i in range(10000):
    try:
        url = fuzzer()
        result = http_program(url)
        print("Success!")
    except ValueError:
        pass

## Mutating Inputs

In [11]:
import random

In [12]:
def delete_random_character(s: str) -> str:
    if s == '':
        return s
    pos = random.randrange(0, len(s))
    return s[:pos] + s[pos + 1:]

In [15]:
seed_input = "A quick brown fox"
for i in range(10):
    x = delete_random_character(seed_input)
    print(repr(x))

'A quic brown fox'
'A uick brown fox'
'A uick brown fox'
'Aquick brown fox'
'A uick brown fox'
'A quik brown fox'
'A quick brwn fox'
' quick brown fox'
' quick brown fox'
'A quick brown ox'


In [16]:
def insert_random_character(s: str) -> str:
    """Returns s with a random character inserted"""
    pos = random.randint(0, len(s))
    random_character = chr(random.randrange(32, 127))
    # print("Inserting", repr(random_character), "at", pos)
    return s[:pos] + random_character + s[pos:]

In [17]:
for i in range(10):
    print(repr(insert_random_character(seed_input)))

'Ap quick brown fox'
'A quicgk brown fox'
'A quNick brown fox'
'A qauick brown fox'
'A qukick brown fox'
'A qjuick brown fox'
'A quick br)own fox'
'A q%uick brown fox'
'A quick br9own fox'
'A quick Tbrown fox'


In [18]:
2^1

3

In [19]:
2^2

0

In [20]:
2 << 3

16

In [21]:
2 >> 1

1

In [22]:
def flip_random_character(s):
    """Returns s with a random bit flipped in a random position"""
    if s == "":
        return s

    pos = random.randint(0, len(s) - 1)
    c = s[pos]
    bit = 1 << random.randint(0, 6)
    new_c = chr(ord(c) ^ bit)
    # print("Flipping", bit, "in", repr(c) + ", giving", repr(new_c))
    return s[:pos] + new_c + s[pos + 1:]

In [23]:
for i in range(10):
    print(repr(flip_random_character(seed_input)))

'A 1uick brown fox'
'A quack brown fox'
'A quick "rown fox'
'A quicc brown fox'
'A quick brown"fox'
'A quick broun fox'
'A quick brkwn fox'
'A quick`brown fox'
'A q}ick brown fox'
'A quick brkwn fox'


In [24]:
def mutate(s: str) -> str:
    """Return s with a random mutation applied"""
    mutators = [
        delete_random_character,
        insert_random_character,
        flip_random_character
    ]
    mutator = random.choice(mutators)
    # print(mutator)
    return mutator(s)

In [25]:
for i in range(10):
    print(repr(mutate("A quick brown fox")))

'A quick brow fox'
'A quik brown fox'
' quick brown fox'
'A quick brownu fox'
'Aquick brown fox'
'A quick brown f/x'
'A quick brovn fox'
'A quick browo fox'
'A quick brown fo'
'A quick br<own fox'


## Mutating URLS

In [26]:
def is_valid_url(url: str) -> bool:
    try:
        result = http_program(url)
        return True
    except ValueError:
        return False

In [27]:
assert is_valid_url("http://www.google.com/search?q=fuzzing")
assert not is_valid_url("xyzzy")

In [28]:
seed_input = "http://www.google.com/search?q=fuzzing"
valid_inputs = set()
trials = 20

for i in range(trials):
    inp = mutate(seed_input)
    if is_valid_url(inp):
        valid_inputs.add(inp)

In [29]:
len(valid_inputs) / trials

0.85

In [30]:
valid_inputs

{'http://www.ggogle.com/search?q=fuzzing',
 'http://www.gooGle.com/search?q=fuzzing',
 'http://www.goog,e.com/search?q=fuzzing',
 'http://www.googl%.com/search?q=fuzzing',
 'http://www.googlOe.com/search?q=fuzzing',
 'http://www.google.co/search?q=fuzzing',
 'http://www.google.com-search?q=fuzzing',
 'http://www.google.com./search?q=fuzzing',
 'http://www.google.com/sarch?q=fuzzing',
 'http://www.google.com/searc?q=fuzzing',
 'http://www.google.com/search?q=fuzzinw',
 'http://www.google.com/searchq=fuzzing',
 'http://www.google.com/suarch?q=fuzzing',
 'http://www.google.co~m/search?q=fuzzing',
 'http://www.google.om/search?q=fuzzing',
 'http://www.googme.com/search?q=fuzzing',
 'http://wwwngoogle.com/search?q=fuzzing'}

## MUltiple Mutations

In [31]:
seed_input = "http://www.google.com/search?q=fuzzing"
mutations = 50

In [32]:
inp = seed_input
for i in range(mutations):
    if i % 5 == 0:
        print(i, "mutations:", repr(inp))
    inp = mutate(inp)

0 mutations: 'http://www.google.com/search?q=fuzzing'
5 mutations: ':http:4//Ywww\\.google.com/seQarch?q=fuzzing'
10 mutations: 'm:http4//Yww\\.google.com/seQarchq=fuLzzing'
15 mutations: 'm:httx4//Ywg\\.google.com/weQarchs=uLzzing'
20 mutations: 'm:tt|4//Y}wg\\.google.com/weQarWcs=uLzzing'
25 mutations: 'm:t|4//Y}wg\\.googe.com/weQar\x17cs=qLz:ing'
30 mutations: 'm:t|4//Y}wg\x1c.WgooXge.com/weQar\x17#s=qLz:in{g'
35 mutations: 'mt|4//Y}wTg\x1c.WgooXge.com/weQar\x17#s=qLz:pi{'
40 mutations: 'mt|4//Y}wT*g\x1c.WgooXge.com:/war\x17#s=qLz:Pi{'
45 mutations: 'mti|4//Y}wT*g\x1c.WgooXglu.c/m:/war\x17#s=qLz:Pi{e'


In [33]:
Fuzzer

ipynb.fs.full.fuzzer.Fuzzer

In [34]:
class MutationFuzzer(Fuzzer):
    """Base class for mutational fuzzing"""

    def __init__(self, seed: List[str],
                 min_mutations: int = 2,
                 max_mutations: int = 10) -> None:
        """Constructor.
        `seed` - a list of (input) strings to mutate.
        `min_mutations` - the minimum number of mutations to apply.
        `max_mutations` - the maximum number of mutations to apply.
        """
        self.seed = seed
        self.min_mutations = min_mutations
        self.max_mutations = max_mutations
        self.reset()

    def reset(self) -> None:
        """Set population to initial seed.
        To be overloaded in subclasses."""
        self.population = self.seed
        self.seed_index = 0

In [35]:
class MutationFuzzer(MutationFuzzer):
    def mutate(self, inp: str) -> str:
        return mutate(inp)

## Guiding by Coverage

In [36]:
class FunctionRunner(Runner):
    def __init__(self, function: Callable) -> None:
        """Initialize.  `function` is a function to be executed"""
        self.function = function

    def run_function(self, inp: str) -> Any:
        return self.function(inp)

    def run(self, inp: str) -> Tuple[Any, str]:
        try:
            result = self.run_function(inp)
            outcome = self.PASS
        except Exception:
            result = None
            outcome = self.FAIL

        return result, outcome

In [37]:
http_runner = FunctionRunner(http_program)
http_runner.run("https://foo.bar/")

(True, 'PASS')

In [5]:
from ipynb.fs.full.coverage import Coverage

SyntaxError: invalid syntax (coverage.ipynb, line 259)